In [1]:
using Pkg
# Pkg.add("Profile")
# Pkg.instantiate()

In [2]:
#= dependencies = ["DelimitedFiles", "LinearAlgebra", "Optim", "Distributions", "Test",
                "BenchmarkTools", "Random", "CSV", "DataFrames", "Distributed", "Plots"];
Pkg.add(dependencies) =#

In [3]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using CSV
using DataFrames
using Plots
using Profile
using Distributed

In [4]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis"

In [5]:
## Include the source code of BulkLMM to be tested:
include("../src/scan.jl");
include("../src/lmm.jl");
include("../src/wls.jl");
include("../src/util.jl");
include("../src/kinship.jl");
include("../src/readData.jl");
include("../src/scan_for_tests.jl")
include("../src/scan_distributed.jl") # new function

## Also include the helper functions for writing tests:
include("../test/testHelper.jl");

In [6]:
## Read in BXD data:
pheno_file = "../data/bxdData/BXDtraits.csv"
pheno = readBXDpheno(pheno_file);
geno_file = "../data/bxdData/BXDgeno_prob.csv"
geno = readGenoProb_ExcludeComplements(geno_file);

kinship = calcKinship(geno); # calculate kinship matrix from genotype data

In [7]:
m = size(pheno, 2);
p = size(geno, 2); # number of markers

In [8]:
## Consider the 7919-th trait
pheno_y = reshape(pheno[:, 7919], :, 1);

In [9]:
n_seeds = 20;

In [17]:
using StatsBase

In [18]:
seeds_list = StatsBase.sample(1:100, n_seeds; replace = false);

In [19]:
(y0, X0, lambda0) = transform1(pheno_y, geno, kinship);

In [20]:
(r0, X00) = transform2(y0, X0, lambda0; reml = false);

In [21]:
# addprocs(10);

In [22]:
workers()

10-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [23]:
# rmprocs(workers())

In [24]:
@everywhere begin
    using Random
    using LinearAlgebra
    using Optim
    using Distributed
    
    ## Include the source code of BulkLMM to be tested:
    include("../src/scan.jl");
    include("../src/lmm.jl");
    include("../src/wls.jl");
    include("../src/util.jl");
    include("../src/kinship.jl");
    include("../src/readData.jl");
    include("../src/scan_for_tests.jl")
    include("../src/scan_distributed.jl")

    include("../src/parallel_helpers.jl")
end

In [25]:
LODs_by_blocks = scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true, 
    option = "by blocks", nblocks = 1000);

In [26]:
LODs_ordinary = scan_perms(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true);

In [27]:
LODs_by_nperms = scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1000, original = true, 
    option = "by nperms", ncopies = 10);

In [28]:
maxSqDiff(LODs_by_blocks, LODs_ordinary)

7.888609052210118e-29

In [29]:
@benchmark scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true, 
    option = "by blocks", nblocks = 120)

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  640.485 ms … 771.505 ms  ┊ GC (min … max): 0.00% … 0.49%
 Time  (median):     691.068 ms               ┊ GC (median):    0.24%
 Time  (mean ± σ):   699.525 ms ±  51.946 ms  ┊ GC (mean ± σ):  0.38% ± 0.50%

  █      ██ █                         ██                    █ █  
  █▁▁▁▁▁▁██▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█ ▁
  640 ms           Histogram: frequency by time          772 ms <

 Memory estimate: 140.51 MiB, allocs estimate: 12933.

In [30]:
@benchmark scan_perms(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.619 s …    3.339 s  ┊ GC (min … max): 4.83% … 7.61%
 Time  (median):     2.979 s               ┊ GC (median):    6.39%
 Time  (mean ± σ):   2.979 s ± 509.577 ms  ┊ GC (mean ± σ):  6.39% ± 1.96%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.62 s         Histogram: frequency by time         3.34 s <

 Memory estimate: 13.30 GiB, allocs estimate: 74635.

In [33]:
@time begin
    for i in 1:1000
        pheno_yi = reshape(pheno[:, i], :, 1);
        scan_perms_distributed(pheno_yi, geno, kinship; reml = false, nperms = 1, rndseed = 0, original = true, 
                               option = "by blocks", nblocks = 120);
    end
end

188.028156 seconds (10.97 M allocations: 27.519 GiB, 0.23% gc time)
